In [42]:
#Common Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [43]:
#Pytorch packages
import torch 
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.autograd import Variable

In [44]:
#loading the dataset
#training data
train_dataset = datasets.MNIST(root='./data',
                              train = True,
                              transform = transforms.ToTensor(),
                              download = True)
#testing dataset
test_dataset = datasets.MNIST(root = './data',
                            train = False,
                            transform = transforms.ToTensor())

In [45]:
batch_size = 100
epochs = 10

In [46]:
#Make the dataset iterable 
#training load
train_load = torch.utils.data.DataLoader(dataset = train_dataset,
                                        batch_size = batch_size,
                                        shuffle = True)
#testing load
test_load = torch.utils.data.DataLoader(dataset = test_dataset,
                                       batch_size = batch_size,
                                       shuffle = False)

In [47]:
#printing the size of the train and test data
print('There are {} images in the training set'.format(len(train_dataset)))
print('There are {} images in the test set'.format(len(test_dataset)))
print('There are {} images in the train loader'.format(len(train_load)))
print('There are {} images in the test loader'.format(len(test_load)))

There are 60000 images in the training set
There are 10000 images in the test set
There are 600 images in the train loader
There are 100 images in the test loader


In [48]:
#Create the model class
class CNN(nn.Module):
    def __init__(self):
        super(CNN,self).__init__()
        self.cnn1 = nn.Conv2d(in_channels=1, out_channels=8, kernel_size=5,stride=1,padding=2)
        #Batch Normalization
        self.batchnorm1 = nn.BatchNorm2d(8)
        #RELU
        self.relu = nn.ReLU()
        self.maxpool1 = nn.MaxPool2d(kernel_size=2)
        self.cnn2 = nn.Conv2d(in_channels=8,out_channels=32,kernel_size=5, stride=1,padding=2)
        #2nd batch
        self.batchnorm2 = nn.BatchNorm2d(32)
        self.relu2 = nn.ReLU()
        self.maxpool2 = nn.MaxPool2d(kernel_size=2)
        #After pooling phase
        self.fc1 = nn.Linear(in_features = 1568, out_features=600)
        self.relu3 = nn.ReLU()
        self.dropout = nn.Dropout(p=0.5)
        self.fc2 = nn.Linear(in_features=600, out_features = 10)
    def forward(self,x):
        out = self.cnn1(x)
        out = self.batchnorm1(out)
        out = self.relu(out)
        out = self.maxpool1(out)
        out = self.cnn2(out)
        out = self.batchnorm2(out)
        out = self.relu2(out)
        out = self.maxpool2(out)
        #flatten the output
        out = out.view(-1,1568)
        out = self.fc1(out)
        out = self.relu(out)
        out = self.fc2(out)
        return out

In [49]:
#using CUDA for this project
model = CNN()
CUDA = torch.cuda.is_available()
if CUDA:
    model = model.cuda()
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr = 0.01)

In [ ]:
#Training the model
iter = 0 
for epoch in range(epochs):
    for i,(images,labels) in enumerate(train_load):
        iter += 1
        if CUDA:
            images = Variable(images.cuda())
            labels = Variable(labels.cuda())
        else:
            images = Variable(images)
            labels = Variable(labels)
            
            
            optimizer.zero_grad()
            outputs = model(images)
            loss = loss_fn(outputs,labels)
            loss.backward()
            optimizer.step()
            #Test the model every 100 iterations. Calculate and print the testing accuracy
            if(i +1) % 100 == 0:
                correct = 0
                total = 0
                for images,labels in test_load:
                    if CUDA:
                        images = Variable(images.cuda())
                    else:
                        images = Variable(images)
                        
                    outputs = model(images)
                    _,predicted = torch.max(outputs.data,1)
                    total += labels.size(0)
                    if CUDA:
                        correct += (predicted.cpu()==labels.cpu()).sum()
                    else:
                        correct += (predicted==labels).sum()
                        
                accuracy = 100 * correct/total
                print('iteration: {}, Train Loss: {}, Test Accuracy: {}%'.format(iter, loss.data[0], accuracy))
print('DONE!')                      